In [ ]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import re, string

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
subm = pd.read_csv('data/sample_submission.csv')

In [3]:
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

C:\Users\User\AppData\Local\Temp\ipykernel_42088\2039795480.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[COMMENT].fillna("unknown", inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_42088\2039795480.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

In [4]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')

def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip(' ')
    return text

def tokenize(text):
    text=clean_text(text)
    return re_tok.sub(r' \1 ', text).split()

In [5]:
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', token_pattern=None,
               use_idf=True, smooth_idf=True, sublinear_tf=True )
train_x = vec.fit_transform(train[COMMENT])
test_x = vec.transform(test[COMMENT])

In [6]:
train_x, test_x

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 17445716 stored elements and shape (159571, 422092)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 14477926 stored elements and shape (153164, 422092)>)

In [7]:
def pr(y_i, y):
    p = train_x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [8]:
def get_mdl(y):
    y = y.values.astype(int) 
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(
        C=4,
        dual=True,
        penalty="l2",
        solver="liblinear",
        max_iter=1000
    )
    x_nb = train_x.multiply(r)
    return m.fit(x_nb, y), r

In [9]:
preds = np.zeros((len(test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit toxic
fit severe_toxic
fit obscene
fit threat
fit insult
fit identity_hate


In [ ]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission.csv', index=False)